In [1]:
# default_exp data

# Data preparation

> Dieses Modul werden die Daten runtergeladen und für die Verarbeitung vorbereitet.

In [1]:
#hide
from nbdev.showdoc import *

## Download und unzip Data

In [2]:
#export
import os
import gdown
import pandas as pd
from pathlib import Path
from zipfile import ZipFile


In [3]:
#export 
def download_santander_data():
    """downloads the data from gdrive to the data folder"""   
    dest = Path('data/raw/train_ver2.csv.zip')
    
    #only download if file not already exists
    if not dest.exists(): 
        dest.parent.mkdir(parents=True, exist_ok=True)
        
        gdown.download(
            'https://drive.google.com/uc?export=download&id=1H-dFcuqI99OkXAawsTMkmylBz3exNUgZ', 
            'data/raw/train_ver2.csv.zip',
            quiet=False)
    else:
        print(f"file {dest} already exists.")
        
    return dest

In [17]:
path_file = download_santander_data()

file data/raw/train_ver2.csv.zip already exists.


In [18]:
#export 
def unzip(path:[Path,str], dest:[Path,str]='.'):
    """unzips a given file"""
    path = Path(path)
    dest = path.parent / path.stem
    
    if not dest.exists():
        with ZipFile(str(path), 'r') as zipObj:
            zipObj.extractall(str(path.parent))
            print(f'extracted to {path.parent / path.stem}')
    else:
        print(f"file {dest} already exists!")
    
    return dest

In [19]:
path_file = unzip(path_file, path_file.parent)

file data/raw/train_ver2.csv already exists!


## clean data
Hierfür verwenden wir im Wesentlichen die Empfehlungen von [Alan Pryor](https://www.kaggle.com/apryor6/detailed-cleaning-visualization-python). 

In [24]:
#export
# laden der Daten in einen DataFrame
def load_csv(path_file:[Path, str], limit_rows=None):
    """load csv file to a pandas df"""
    
    if limit_rows is None:  
        return pd.read_csv(
            path_file,
            dtype={
                "sexo":str,
                "ind_nuevo":str,
                "ult_fec_cli_1t":str,
                "indext":str})
    
    return pd.read_csv(
        path_file,
        dtype={
            "sexo":str,
            "ind_nuevo":str,
            "ult_fec_cli_1t":str,
            "indext":str},
        nrows=limit_rows
    )

In [26]:
df = load_csv(path_file, 1000000)

/home/wilhelm/miniconda3/envs/CommunityLearning/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [ ]:
#export
def clean_dates(df):
    """converting the dates of the df"""
    df["fecha_dato"] = pd.to_datetime(df["fecha_dato"],format="%Y-%m-%d")
    df["fecha_alta"] = pd.to_datetime(df["fecha_alta"],format="%Y-%m-%d")

In [ ]:
#export 
def add_month_feature(df):
    """add buy month to the features"""
    df["month"] = pd.DatetimeIndex(df["fecha_dato"]).month
    return df

def add_age_feature(df):
    """add customer age feature"""
    df["age"]   = pd.to_numeric(df["age"], errors="coerce")
    return df

In [ ]:
# bereinigen 